In [94]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from google.cloud import bigquery
import time
%env GOOGLE_APPLICATION_CREDENTIALS=../secrets/bigquery-service-account.json 
client = bigquery.Client()
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

env: GOOGLE_APPLICATION_CREDENTIALS=../secrets/bigquery-service-account.json


In [95]:
%load_ext google.cloud.bigquery

The google.cloud.bigquery extension is already loaded. To reload it, use:
  %reload_ext google.cloud.bigquery


In [96]:
%%bigquery metadata --project masterarbeit-245718 --verbose 
SELECT * FROM `bigquery-public-data`.crypto_ethereum.INFORMATION_SCHEMA.COLUMNS where table_name = "traces"

Executing query with job ID: 776c01d7-6acb-4e31-8261-864f4e9ca27c
Query executing: 4.76s
Query complete after 5.43s


In [97]:
size_sample_data = 200

In [98]:
metadata.to_html("./test.html")

### Generate sample data for "call_type"

In [99]:
# %%bigquery call_types_sql_result --project masterarbeit-245718 --verbose 
# select DISTINCT call_type from `bigquery-public-data.crypto_ethereum.traces`
# where DATE(block_timestamp) >= '2019-07-6' AND DATE(block_timestamp) <= '2019-7-7'

In [100]:
call_types = ['call', 'delegatecall', 'staticcall', 'callcode', None]
prob_call_types = [0.7, 0.05, 0.025, 0.025, 0.2]

In [101]:
call_type_sample = np.random.choice(call_types, 2*size_sample_data, p=prob_call_types)

# Generate sample data for "trace_type"

In [102]:
additional_trace_types = ["create", "suicide", "reward", "genesis", "daofork"]
prob_additional_trace_types = [0.025, 0.025, 0.9, 0.025, 0.025]

trace_type_sample = []
for ct in call_type_sample:
    if ct == None:
        trace_type_sample.append(np.random.choice(additional_trace_types, p=prob_additional_trace_types))
    else:
        trace_type_sample.append("call")

### Generate sample data for "status"

In [103]:
# %%bigquery status_values --project masterarbeit-245718 --verbose 
# select DISTINCT status from `bigquery-public-data.crypto_ethereum.traces`
# where DATE(block_timestamp) >= '2019-07-6' AND DATE(block_timestamp) <= '2019-7-7'

In [104]:
status_values = [0, 1]
probs_status_values = [0.05, 0.95]

In [105]:
probs_status_values = [0.05, 0.95]

In [106]:
status_sample = np.random.choice(status_values, 2*size_sample_data, p=probs_status_values)

### Generate sample accounts 

In [107]:
exchanges = ["exchange_{}".format(i) for i in range(1,int(0.05*size_sample_data + 1))]

In [108]:
speculators = ["speculator_{}".format(i) for i in range(1,int(0.95*size_sample_data + 1))]

In [109]:
sample_addresses = []
sample_addresses.extend(speculators)
sample_addresses.extend(exchanges)

In [110]:
sample_addresses = pd.DataFrame(sample_addresses, columns=["address"])

## Upload 'addresses' table to bigquery 

In [111]:
sample_addresses.to_gbq('ethereum_us.sample_addresses', if_exists="replace")

1it [00:04,  4.30s/it]


### Generate sample transactions (speculators to exchanges)

In [112]:
from_spec_addresses = [np.random.choice(speculators) for i in range(int(2*size_sample_data/2))]
to_ex_addresses = [np.random.choice(exchanges) for i in range(int(2*size_sample_data/2))]
values_spec_to_ex = np.random.randint(1, 20, int(2*size_sample_data/2))

In [113]:
txdata1 = pd.DataFrame(zip(from_spec_addresses, to_ex_addresses, values_spec_to_ex), columns=["from_address", "to_address", "value"])

### Generate sample transactions (exchanges to speculators)

In [114]:
to_spec_addresses = [np.random.choice(speculators) for i in range(int(2*size_sample_data/2))]
from_ex_addresses = [np.random.choice(exchanges) for i in range(int(2*size_sample_data/2))]
values_ex_to_spec = np.random.randint(1, 5, int(2*size_sample_data/2))

In [115]:
txdata2 = pd.DataFrame(zip(from_ex_addresses, to_spec_addresses, values_ex_to_spec), columns=["from_address", "to_address", "value"])

Anmerkung: Die speculators schicken mehr Geld zu den Börsen als umgekehrt.

In [116]:
txdata = txdata1.append(txdata2).reset_index(drop=True)
txdata = txdata.sample(frac=1)

### Generate sample "block_timestamps"

In [117]:
import datetime as datetime

base = datetime.datetime.strptime("2020-02-02", '%Y-%m-%d')
block_timestamps = [base - datetime.timedelta(seconds=x) for x in range(0, 12*size_sample_data, 12)]
base = base - datetime.timedelta(days=31)
block_timestamps.extend([base - datetime.timedelta(seconds=x) for x in range(0, 12*size_sample_data, 12)])

### Merge data to sample "traces" table

In [118]:
sample_traces = txdata.copy()

In [119]:
sample_traces["status"] = status_sample 

In [120]:
sample_traces["call_type"] = call_type_sample 

In [121]:
sample_traces["trace_type"] = trace_type_sample 

In [122]:
sample_traces["block_timestamp"] = block_timestamps 

In [123]:
sample_traces.sort_values(by="from_address")

,from_address,to_address,value,status,call_type,trace_type,block_timestamp
385,exchange_1,speculator_140,2,1,staticcall,call,2020-01-01 23:53:36
236,exchange_1,speculator_65,2,1,call,call,2020-01-01 23:55:48
248,exchange_1,speculator_97,2,1,call,call,2020-01-01 23:57:48
287,exchange_1,speculator_135,4,1,call,call,2020-01-01 23:58:24
309,exchange_1,speculator_82,3,1,call,call,2020-01-01 23:41:24
246,exchange_1,speculator_93,1,1,delegatecall,call,2020-02-01 23:36:36
325,exchange_1,speculator_180,2,1,None,reward,2020-01-01 23:37:48
376,exchange_1,speculator_6,3,1,call,call,2020-02-01 23:57:24
277,exchange_1,speculator_122,2,1,call,call,2020-02-01 23:57:48
251,exchange_1,speculator_159,1,1,call,call,2020-01-01 23:36:12


### Upload "traces" table to bigquery

In [124]:
sample_traces.to_gbq('ethereum_us.sample_traces', if_exists="replace")

1it [00:03,  3.89s/it]


### Test SQL command for retrieving "weiReceived", "weiSent"

In [125]:
%%bigquery res1 --project masterarbeit-245718 --verbose 
with weiView as (

  with weiReceivedView as (
        
      -- debits
      select to_address, sum(ifnull(value, 0)) as weiReceived
      from `ethereum_us.sample_traces` 
      where to_address is not null
      and status = 1
      and (call_type not in ('delegatecall', 'callcode', 'staticcall') or call_type is null) 
      group by to_address
        
  ), weiSentView as (
  
      -- credits
      select from_address, sum(ifnull(value, 0)) as weiSent
      from  `ethereum_us.sample_traces` 
      where from_address is not null
      and status = 1
      and (call_type not in ('delegatecall', 'callcode', 'staticcall') or call_type is null) 
      group by from_address
  ) 
  select 
  CASE 
    when to_address is not null then to_address
    when from_address is not null then from_address
  end as address, 
  ifnull(weiReceived,0) as weiReceived, 
  ifnull(weiSent,0) as weiSent
  from weiReceivedView full outer join weiSentView on from_address = to_address
) 
select address, weiReceived, weiSent from weiView right join `ethereum_us.sample_addresses`  using(address)

Executing query with job ID: 1a604709-1a79-4c9b-9c87-e7959e4036b2
Query executing: 1.61s
Query complete after 2.42s


In [126]:
res2 = res1.copy()
res2["balance"] = res1.weiReceived - res1.weiSent
res2 = res2.set_index("address")
res2 = res2.sort_values(by="address", ascending=False)
res2 = res2.astype("float")
res2 = res2.fillna(0.)
balance_result_sql = res2

In [127]:
%%bigquery sample_traces --project masterarbeit-245718 --verbose 
select * from `ethereum_us.sample_traces`

Executing query with job ID: 6ccbee1d-5a8d-4701-9e88-26b52ee85409
Query executing: 0.97s
Query complete after 2.20s


In [128]:
data1 = [row for (index, row) in sample_traces.iterrows() if (row.call_type not in ['delegatecall', 'callcode', 'staticcall'] or row.call_type == None) and row.status == 1]
data1 = pd.DataFrame(data1)

In [129]:
data2 = pd.DataFrame(index=set(data1["from_address"].unique()) | set(data1["to_address"].unique()))
data2["weiReceived"] = data1.groupby('to_address').sum().value
data2["weiSent"] = data1.groupby('from_address').sum().value
data2["weiSent"] = data2["weiSent"].fillna(0.)
data2["weiReceived"] = data2["weiReceived"].fillna(0.)
data2["balance"] = data2["weiReceived"] - data2["weiSent"]
data2.index = data2.index.rename("address")
data2 = data2.reindex(sample_addresses["address"], fill_value=0.)
data2 = data2.sort_values(by="address", ascending=False)
balance_result_py = data2

In [130]:
balance_result_py.head()
balance_result_sql.head()

,weiReceived,weiSent,balance
address,,,
speculator_99,0.0,29.0,-29.0
speculator_98,0.0,0.0,0.0
speculator_97,4.0,0.0,4.0
speculator_96,8.0,0.0,8.0
speculator_95,0.0,0.0,0.0


,weiReceived,weiSent,balance
address,,,
speculator_99,0.0,29.0,-29.0
speculator_98,0.0,0.0,0.0
speculator_97,4.0,0.0,4.0
speculator_96,8.0,0.0,8.0
speculator_95,0.0,0.0,0.0


In [131]:
pd.testing.assert_frame_equal(balance_result_py, balance_result_sql)
print("weiSent, weiReceived Test succeeded!!")

weiSent, weiReceived Test succeeded!!


### Test SQL command for retrieving "txSent" and "txReceived"

In [132]:
%%bigquery res2 --project masterarbeit-245718 --verbose 
with txView as (

  with txSent as (
      SELECT from_address, count(*) as numberOfTranscationsSent FROM `masterarbeit-245718.ethereum_us.sample_traces` 
      where to_address is not null and status = 1 and (call_type not in ('delegatecall', 'callcode', 'staticcall') or call_type is null) 
      group by from_address
    ), txReceived as (
      SELECT to_address, count(*) as numberOfTranscationsReceived FROM `masterarbeit-245718.ethereum_us.sample_traces` 
      where to_address is not null and status = 1 and (call_type not in ('delegatecall', 'callcode', 'staticcall') or call_type is null) 
      group by to_address
    ) 
    SELECT 
    CASE  
      WHEN to_address IS NOT NULL THEN to_address
      WHEN from_address IS NOT NULL THEN from_address
    END AS address,
    IFNULL(numberOfTranscationsReceived, 0) as numberOfTranscationsReceived, 
    IFNULL(numberOfTranscationsSent, 0) as numberOfTranscationsSent
    from txReceived FULL OUTER JOIN txSent on to_address = from_address
) 

    select address, numberOfTranscationsReceived, numberOfTranscationsSent from txView right join `ethereum_us.sample_addresses` using(address)

Executing query with job ID: 12ee77eb-9650-4473-af02-28b6563e56ca
Query executing: 1.41s
Query complete after 2.59s


In [133]:
res3 = res2.copy()
res3 = res3.fillna(0.)
res3 = res3.sort_values(by="address", ascending=False)
tx_sent_received_result_sql = res3.set_index("address", drop=True)

In [134]:
data3 = pd.DataFrame(index=set(data1["from_address"].unique()) | set(data1["to_address"].unique()))
data3["numberOfTranscationsReceived"] = data1.groupby('to_address').count().value
data3["numberOfTranscationsSent"] = data1.groupby('from_address').count().value
data3 = data3.fillna(0)
data3 = data3.astype("int")
data3.index = data3.index.rename("address")
data3 = data3.reindex(sample_addresses["address"], fill_value=0.)
data3 = data3.sort_values(by="address", ascending=False)
tx_sent_received_result_py = data3

In [135]:
pd.testing.assert_frame_equal(tx_sent_received_result_py, tx_sent_received_result_sql)
print("txSent, txReceived Test succeeded !!")

txSent, txReceived Test succeeded !!


In [136]:
features = balance_result_sql.join(tx_sent_received_result_sql,how="left")
features = features.reset_index()
features.to_gbq('ethereum_us.sample_features', if_exists="replace")

1it [00:03,  3.48s/it]


# avg_wei_sent, avg_wei_recd

In [137]:
%%bigquery --project masterarbeit-245718 --verbose 

select 
    address, weiReceived, weiSent, balance, numberOfTranscationsReceived, numberOfTranscationsSent,
    CASE 
        when numberOfTranscationsSent > 0 THEN weiSent / numberOfTranscationsSent 
        when numberOfTranscationsSent = 0 THEN 0 
    END as agv_wei_sent,
    CASE 
        when numberOfTranscationsReceived > 0 THEN weiReceived / numberOfTranscationsReceived 
        when numberOfTranscationsReceived = 0 THEN 0 
    END as avg_wei_recd
from `ethereum_us.sample_features`

Executing query with job ID: 137623fb-5e7b-4200-b45b-309c6f04ec0b
Query executing: 0.77s
Query complete after 1.45s


,address,weiReceived,weiSent,balance,numberOfTranscationsReceived,numberOfTranscationsSent,agv_wei_sent,avg_wei_recd
0,speculator_98,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
1,speculator_95,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
2,speculator_94,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
3,speculator_92,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
4,speculator_91,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
5,speculator_88,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
6,speculator_84,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
7,speculator_78,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
8,speculator_74,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
9,speculator_67,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000


#### Test: avg_wei_sent, avg_wei_recd

Benötigt keine Tests, da der SQL Befehl sehr einfach aufgebaut ist.

### Test avg time between tx

In [138]:
%%bigquery res4 --project masterarbeit-245718 --verbose 
with timeRecView as (

  with receivedTx as (
    SELECT to_address, count(*) as numberOfTranscationsReceived FROM `masterarbeit-245718.ethereum_us.sample_traces` 
    where to_address is not null 
      and status = 1 
      and (call_type not in ('delegatecall', 'callcode', 'staticcall') or call_type is null)
    group by to_address),
  
  timeStampDiffs as (
    SELECT to_address, TIMESTAMP_DIFF(MAX(block_timestamp), MIN( block_timestamp ), second ) as timestampDiff
    FROM `masterarbeit-245718.ethereum_us.sample_traces`
    where to_address is not null 
      and status = 1 
      and (call_type not in ('delegatecall', 'callcode', 'staticcall') or call_type is null)
    group by to_address
  
  ) select to_address as address, 
  CASE 
    when (numberOfTranscationsReceived - 1)  > 0 then timestampDiff / (numberOfTranscationsReceived - 1) 
    else 0
  end as avgTimeDiffBetweenReceivedTransactions
  from receivedTx inner join  timeStampDiffs using(to_address)
)

select address, ifnull(avgTimeDiffBetweenReceivedTransactions,0) as avgTimeDiffBetweenReceivedTransactions from timeRecView right join `ethereum_us.sample_addresses` using(address)

Executing query with job ID: dd9def29-7e11-4e0b-98c1-719e6a9092c6
Query executing: 1.56s
Query complete after 2.59s


In [139]:
res6 = res4.set_index("address", drop=True)
res6 = res6.fillna(0.)
res6 = res6.sort_values(by="address")
avg_time_diff_receivedtx_result_sql = res6

In [140]:
%%bigquery res5 --project masterarbeit-245718 --verbose 
with timeSentView as (

  with sentTx as (
    SELECT from_address, count(*) as numberOfTranscationsSent FROM `masterarbeit-245718.ethereum_us.sample_traces` 
    where to_address is not null 
      and status = 1 
      and (call_type not in ('delegatecall', 'callcode', 'staticcall') or call_type is null)
    group by from_address),
  timeStampDiffs as (
    SELECT from_address, TIMESTAMP_DIFF(MAX(block_timestamp), MIN( block_timestamp ), second ) as timestampDiff
    FROM `masterarbeit-245718.ethereum_us.sample_traces`
    where to_address is not null 
      and status = 1 
      and (call_type not in ('delegatecall', 'callcode', 'staticcall') or call_type is null)
    group by from_address
  ) select from_address as address, 
  CASE 
    when (numberOfTranscationsSent - 1)  > 0 then timestampDiff / (numberOfTranscationsSent - 1) 
    else 0
  end as avgTimeDiffBetweenSentTransactions
     from sentTx inner join  timeStampDiffs using(from_address)
)

select address, ifnull(avgTimeDiffBetweenSentTransactions,0) as avgTimeDiffBetweenSentTransactions from timeSentView right join `ethereum_us.sample_addresses` using(address)

Executing query with job ID: 481a89b2-439a-44fb-a24f-41afa208a2f2
Query executing: 1.59s
Query complete after 2.40s


In [141]:
res6 = res5.set_index("address", drop=True)
res6 = res6.fillna(0.)
res6 = res6.sort_values(by="address")
avg_time_diff_senttx_result_sql = res6

In [142]:
res7 = [row for (index, row) in sample_traces.iterrows() if (row.call_type not in ['delegatecall', 'callcode', 'staticcall'] or row.call_type == None) and row.status == 1]
res7 = pd.DataFrame(res7)
res8 = res7.groupby("from_address").max().block_timestamp
res9 = res7.groupby("from_address").min().block_timestamp
res10 = res8 - res9
res10 = res10.rename("seconds_diff")
res10 = tx_sent_received_result_py.join(res10, how="right").drop("numberOfTranscationsReceived", axis=1)
res10 = res10.fillna(0.)
res10["avgTimeDiffBetweenSentTransactions"] = res10["seconds_diff"] / (res10["numberOfTranscationsSent"] - 1)
res10["avgTimeDiffBetweenSentTransactions"] = res10["avgTimeDiffBetweenSentTransactions"].fillna(datetime.timedelta(0))
res10 = res10.reindex(sample_addresses["address"], fill_value=0.)
res10.index = res10.index.rename("address")
res10 = res10.sort_values(by="address")
res10 = res10.drop(["numberOfTranscationsSent", "seconds_diff"],axis=1)
res10["avgTimeDiffBetweenSentTransactions"] = [ts.total_seconds() for ts in res10["avgTimeDiffBetweenSentTransactions"]]
avg_time_diff_senttx_result_py = res10

/home/mgsgde/.local/lib/python3.6/site-packages/pandas/core/arrays/timedeltas.py:523: RuntimeWarning: invalid value encountered in true_divide
  result = self._data / other


In [143]:
pd.testing.assert_frame_equal(avg_time_diff_senttx_result_sql, avg_time_diff_senttx_result_py)
print("avgTimeDiffBetweenSentTransactions Test succeeded !!")

avgTimeDiffBetweenSentTransactions Test succeeded !!


In [144]:
features = balance_result_sql.join(tx_sent_received_result_sql,how="left")
features = features.join(avg_time_diff_senttx_result_sql)
features = features.join(avg_time_diff_receivedtx_result_sql)
features = features.sort_values(by="balance", ascending=False)
features = features.fillna(0.0)
import sys
# addresses that have sent/received only one transaction get the avg time max * 2
features["avgTimeDiffBetweenSentTransactions"] = features["avgTimeDiffBetweenSentTransactions"].replace(to_replace=0.0, value=2 * max(features["avgTimeDiffBetweenSentTransactions"]))
features["avgTimeDiffBetweenReceivedTransactions"] = features["avgTimeDiffBetweenReceivedTransactions"].replace(to_replace=0.0, value=2 * max(features["avgTimeDiffBetweenReceivedTransactions"]))

In [145]:
features = features.reset_index()

In [146]:
features.to_gbq('ethereum_us.sample_features', if_exists="replace")

1it [00:03,  3.62s/it]


# Test mined Blocks

In [147]:
%%bigquery sql_res_minedblocks --project masterarbeit-245718 --verbose 

with minedBlocksView as (
    SELECT to_address as address, count(*) as mined_blocks FROM `masterarbeit-245718.ethereum_us.sample_traces`
        where trace_type = "reward"
        group by to_address
    )
select address, ifnull(mined_blocks,0) as minedBlocks from minedBlocksView right join `ethereum_us.sample_addresses` using(address)


Executing query with job ID: 25db5d97-82f8-41b2-aa4f-479ff9bc0a67
Query executing: 1.14s
Query complete after 2.36s


In [148]:
features.head()

,address,weiReceived,weiSent,balance,numberOfTranscationsReceived,numberOfTranscationsSent,avgTimeDiffBetweenSentTransactions,avgTimeDiffBetweenReceivedTransactions
0,exchange_2,233.0,46.0,187.0,19.0,17.0,167520.750000,148885.333333
1,exchange_7,215.0,39.0,176.0,22.0,17.0,167475.000000,127629.142857
2,exchange_3,235.0,63.0,172.0,20.0,22.0,127652.571429,141061.894737
3,exchange_10,162.0,26.0,136.0,15.0,13.0,223350.000000,191460.857143
4,exchange_9,160.0,39.0,121.0,16.0,14.0,206165.538462,178688.800000


# Test var(avgTimeDiff...)

In [149]:
%%bigquery res_sql_stddev --project masterarbeit-245718 --verbose 

with timestamp_var as (
    with timestamps_diffs as (
        
        with timestamps_preceding_tx as (
            
            with timestamps_sent_tx as (
                select from_address, block_timestamp from `masterarbeit-245718.ethereum_us.sample_traces`
                    where from_address is not null 
                      and status = 1 
                      and (call_type not in ('delegatecall', 'callcode', 'staticcall') or call_type is null)
            )
            
            select from_address, block_timestamp,
                lag(block_timestamp) OVER (partition by from_address order by block_timestamp asc) as preceding_block_timestamp 
            from timestamps_sent_tx
        )
        
        select from_address, block_timestamp, preceding_block_timestamp, 
            TIMESTAMP_DIFF(block_timestamp, preceding_block_timestamp, second) as timestampdiff
        from timestamps_preceding_tx
    )
    select from_address as address, STDDEV_SAMP(timestampdiff) as stddev_sample  
    from timestamps_diffs group by from_address 
) 
select * from timestamp_var right join `ethereum_us.sample_addresses` using(address)

Executing query with job ID: 09508986-f3b2-4ab5-ae7d-002ba00b97cd
Query executing: 1.57s
Query complete after 2.29s


In [150]:
stddev_result_sql = res_sql_stddev.set_index("address")
stddev_result_sql = stddev_result_sql.reindex(sample_addresses["address"])
stddev_result_sql = stddev_result_sql.fillna(res_sql_stddev["stddev_sample"].max())
stddev_result_sql = stddev_result_sql.sort_values(by="address")

In [151]:
%%bigquery res_sql_stddev --project masterarbeit-245718 --verbose 

with timestamp_var as (
    
    with timestamps_diffs as (
        
        with timestamps_preceding_tx as (
            
            with timestamps_sent_tx as (
                select to_address, block_timestamp from `masterarbeit-245718.ethereum_us.sample_traces`
                    where to_address is not null 
                      and status = 1 
                      and (call_type not in ('delegatecall', 'callcode', 'staticcall') or call_type is null)
            )
            
            select to_address, block_timestamp,
                lag(block_timestamp) OVER (partition by to_address order by block_timestamp asc) as preceding_block_timestamp 
            from timestamps_sent_tx
        )
        
        select to_address, block_timestamp, preceding_block_timestamp, 
            TIMESTAMP_DIFF(block_timestamp, preceding_block_timestamp, second) as timestampdiff
        from timestamps_preceding_tx
    )
    select to_address as address, STDDEV_SAMP(timestampdiff) as stddev_sample  
    from timestamps_diffs group by to_address 
) 
select * from timestamp_var right join `ethereum_us.sample_addresses` using(address)

Executing query with job ID: e9909e01-ece2-413a-a1c5-f59b3914e167
Query executing: 0.86s
Query complete after 1.95s


In [152]:
stddev_result_sql = res_sql_stddev.set_index("address")
stddev_result_sql = stddev_result_sql.reindex(sample_addresses["address"])
stddev_result_sql = stddev_result_sql.fillna(res_sql_stddev["stddev_sample"].max())
stddev_result_sql = stddev_result_sql.sort_values(by="stddev_sample")

# Verifikation der var(avgTimeDiff...) Tabelle via Python

In [153]:
res7 = [row for (index, row) in sample_traces.iterrows() if (row.call_type not in ['delegatecall', 'callcode', 'staticcall'] or row.call_type == None) and row.status == 1]
res7 = pd.DataFrame(res7)
res7 = res7.sort_values(by="block_timestamp", ascending=True)

In [154]:
def calculateDifference(df):
#     from_address = df["from_address"].reset_index(drop=True)[0]
    variance = df["block_timestamp"].diff().std()
    return variance
#     return pd.DataFrame({"variance": [variance]})

res8 = res7.groupby("to_address").apply(lambda df: df["block_timestamp"].diff().std() )
res8 = pd.DataFrame(res8, columns=["stddev_sample"])
res8 = res8.reindex(sample_addresses["address"])
res8 = res8.fillna(res8.max())
res8["stddev_sample"] = res8["stddev_sample"] / np.timedelta64(1, 's')
res8 = res8.sort_values(by="stddev_sample")
stddev_result_py = res8

In [155]:
# pd.testing.assert_frame_equal(stddev_result_py, stddev_result_sql)
np.testing.assert_almost_equal(list(stddev_result_py["stddev_sample"]), list(stddev_result_sql["stddev_sample"]), decimal=2)
print("stddev Test succeeded !!")

stddev Test succeeded !!


# Erstellen der traces_usd Tabelle

In [156]:
import os 
from datetime import datetime

In [157]:
eth_usd_res = pd.read_csv('../data/eth_usd.csv') 
# Preview the first 5 lines of the loaded data 
eth_usd = eth_usd_res.copy()
eth_usd["avg_usd_eth"] = (eth_usd_res["High"] + eth_usd_res["Low"]) / 2
eth_usd["usd_eth_timestamp"] = [datetime.strptime(ts, "%Y-%m-%d") for ts in eth_usd_res["Date"]]
eth_usd = eth_usd.loc[:, ["usd_eth_timestamp", "avg_usd_eth"]]
# data.sort_values(by="avg", ascending=False)
# data.iloc[data["avg_usd_eth"].idxmax(),:]
eth_usd.tail()

,usd_eth_timestamp,avg_usd_eth
1645,2020-02-07,217.722564
1646,2020-02-08,220.987190
1647,2020-02-09,226.423241
1648,2020-02-10,223.632332
1649,2020-02-11,222.864899


In [158]:
eth_usd.to_gbq('ethereum_us.usd_eth', if_exists="replace")

1it [00:04,  4.36s/it]


In [159]:
%%bigquery traces_usd --project masterarbeit-245718 --verbose 

select from_address, to_address, value, status, call_type, trace_type, block_timestamp, avg_usd_eth * value as value_usd from `masterarbeit-245718.ethereum_us.sample_traces` as traces left join `masterarbeit-245718.ethereum_us.usd_eth` as usd_eth 
  on (TIMESTAMP_TRUNC(usd_eth.usd_eth_timestamp, DAY, 'UTC') = TIMESTAMP_TRUNC(traces.block_timestamp, DAY, 'UTC'))

Executing query with job ID: 86dbbc34-087d-483c-a24b-f5cf0d577dd1
Query executing: 0.95s
Query complete after 2.01s


In [160]:
traces_usd.to_gbq('ethereum_us.sample_traces_usd', if_exists="replace")

1it [00:04,  4.70s/it]


# Total USD received / sent 

Analog zu wei.

# AVG USD received / sent

Analog zu wei.

# Montly USD reveived / sent

Analog zu wei.

# Active Months

In [161]:
%%bigquery active_months_result_sql --project masterarbeit-245718 --verbose 

with traces_clean as (
    select * from `masterarbeit-245718.ethereum_us.sample_traces` where 
    status = 1 and (call_type not in ('delegatecall', 'callcode', 'staticcall') or call_type is null) 
    
), tx_received as (
    select 
        TIMESTAMP_TRUNC(block_timestamp, MONTH, 'UTC') as month, 
        to_address,
        count(*) as number_tx_received
    from traces_clean
    group by TIMESTAMP_TRUNC(block_timestamp, MONTH, 'UTC'), to_address
    
), tx_sent as (
    select 
        TIMESTAMP_TRUNC(block_timestamp, MONTH, 'UTC') as month, 
        from_address,
        count(*) as number_tx_sent
    from traces_clean
    group by TIMESTAMP_TRUNC(block_timestamp, MONTH, 'UTC'), from_address
    
), monthly_tx as (
    select 
        CASE  
          WHEN tx_sent.from_address IS NOT NULL THEN tx_sent.from_address
          WHEN tx_received.to_address IS NOT NULL THEN tx_received.to_address
        END AS address,
        CASE  
          WHEN tx_sent.month IS NOT NULL THEN tx_sent.month
          WHEN tx_received.month IS NOT NULL THEN tx_received.month
        END AS month,
        ifnull(number_tx_sent,0) as number_tx_sent, 
        ifnull(number_tx_received,0) as number_tx_received 
    from tx_sent full join tx_received 
        on (tx_sent.from_address = tx_received.to_address and tx_sent.month = tx_received.month)

), active_months_view as (
    select 
        address, 
        countif(number_tx_sent > 0 or number_tx_received > 0) as active_months 
    from monthly_tx group by address order by address ASC 
)

select 
    address, 
    ifnull(active_months,0) as active_months 
from ethereum_us.sample_addresses left join active_months_view using(address)



Executing query with job ID: 0ab67569-0f8e-40fc-85d6-6db8761d9bae
Query executing: 1.43s
Query complete after 2.86s


#### Tests für active_months feature

In [162]:
traces = [row for (index, row) in sample_traces.iterrows() if (row.call_type not in ['delegatecall', 'callcode', 'staticcall'] or row.call_type == None) and row.status == 1]
traces = pd.DataFrame(traces)

In [163]:
def address_was_active(address, month):
    for i,t in traces.iterrows():
        if (t["from_address"] == address or t["to_address"] == address) and (month.strftime("%Y-%m") == t["block_timestamp"].strftime("%Y-%m")):
            return True
    return False
        

In [164]:
from datetime import timedelta

active_months = {}

min_ts = traces["block_timestamp"].min()
max_ts = traces["block_timestamp"].max()
min_ts = min_ts.to_pydatetime()
max_ts = max_ts.to_pydatetime()

         
months = [month.to_pydatetime() for month in pd.date_range(start=min_ts, end=(max_ts + timedelta(days=31)), freq="MS")]

for index,address in sample_addresses.iterrows():
    for month in months:
        if (address_was_active(address[0], month)):
            active_months[address[0]] = active_months[address[0]] + 1 if address[0] in active_months else 1 

In [165]:
s = pd.Series(active_months)
df = pd.DataFrame(s)
df = df.reindex(sample_addresses["address"])
df = df.reset_index()
df.columns=["address", "active_months"]
df = df.fillna(0.)
df["active_months"] = df["active_months"].astype(int)
active_months_result_py  = df
pd.testing.assert_frame_equal(active_months_result_py, active_months_result_sql)
print("active months Test succeeded!!")

active months Test succeeded!!


# montly_wei_sent, monthly_wei_recd

In [166]:
active_months_result_sql = active_months_result_sql.set_index("address")
features = balance_result_sql.join(active_months_result_sql, how="left")

In [167]:
features.to_gbq('ethereum_us.sample_features', if_exists="replace")

1it [00:04,  4.92s/it]


In [171]:
%%bigquery --project masterarbeit-245718 --verbose 

select 
    *,
    CASE 
        when active_months > 0 THEN weiSent / active_months 
        when active_months = 0 THEN 0 
    END as monthl_wei_sent,
    CASE 
        when active_months > 0 THEN weiReceived / active_months 
        when active_months = 0 THEN 0 
    END as avg_wei_recd
from `ethereum_us.sample_features`

Executing query with job ID: b8417c49-999e-4a77-9256-a08b7f63f7de
Query executing: 1.48s
Query complete after 3.54s


,weiReceived,weiSent,balance,active_months,agv_wei_sent,avg_wei_recd
0,0.0,0.0,0.0,0,0.0,0.0
1,0.0,0.0,0.0,0,0.0,0.0
2,0.0,0.0,0.0,0,0.0,0.0
3,0.0,0.0,0.0,0,0.0,0.0
4,0.0,0.0,0.0,0,0.0,0.0
5,0.0,0.0,0.0,0,0.0,0.0
6,0.0,0.0,0.0,0,0.0,0.0
7,0.0,0.0,0.0,0,0.0,0.0
8,0.0,0.0,0.0,0,0.0,0.0
9,0.0,0.0,0.0,0,0.0,0.0


# monthly_outgoing_txns, monthly_incoming_txns

In [173]:
features = tx_sent_received_result_sql.join(active_months_result_sql, how="left")

In [174]:
features.to_gbq('ethereum_us.sample_features', if_exists="replace")

1it [00:04,  4.30s/it]


In [ ]:
%%bigquery --project masterarbeit-245718 --verbose 

select 
    *,
    CASE 
        when active_months > 0 THEN numberOfTranscationsSent / active_months 
        when active_months = 0 THEN 0 
    END as agv_tx_sent,
    CASE 
        when active_months > 0 THEN numberOfTranscationsReceived / active_months 
        when active_months = 0 THEN 0 
    END as avg_wei_recd
from `ethereum_us.sample_features`

# Deprecated

In [168]:
# %%bigquery res6 --project masterarbeit-245718 --verbose 

# SELECT
# * 
# FROM
# `masterarbeit-245718.ethereum_us.INFORMATION_SCHEMA.TABLES`


In [169]:
# tablesToDelete = [tn for tn in res6["table_name"] if tn.startswith("top40k_19_20")]

# for t in tablesToDelete:
#      client.delete_table("ethereum_us.{}".format(t), not_found_ok=True)  

In [170]:
# view_ref = shared_dataset_ref.table("my_shared_view")
# view = bigquery.Table(view_ref)
# sql_template = 'SELECT name, post_abbr FROM `{}.{}.{}` WHERE name LIKE "W%"'
# view.view_query = sql_template.format(project, source_dataset_id, source_table_id)
# view = client.create_table(view)  # API request

# print("Successfully created view at {}".format(view.full_table_id))